# Семинар 10-11: Деревья Решений и Случайный Лес на практике

**Цель семинара:** Показать полный цикл работы с древовидными моделями на двух разных типах задач: классификации и регрессии. Сделать акцент на визуализации, интерпретации и сравнении одиночного дерева с ансамблем.


## Часть 1: Задача Классификации

**Датасет:** `penguins_size.csv` — данные о пингвинах. Простой и чистый, отлично подходит для визуализации.
**Цель:** Научиться предсказывать вид пингвина (`species`).

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.read_csv('https://raw.githubusercontent.com/yuliya-sabirova/ml-course/main/data/penguins_size.csv')
# Удаляем строки с пропусками для простоты
df = df.dropna()
df.info()
df.head()

### 1.1. Анализ и подготовка данных

**Комментарий:** *Посмотрим на данные. `pairplot` отлично показывает, как признаки соотносятся друг с другом в разрезе классов. Видно, что некоторые пары признаков почти идеально разделяют классы, что является хорошим знаком для древовидных моделей.*

In [ ]:
sns.pairplot(df, hue='species')
plt.show()

**Комментарий:** *Деревьям нужны числовые данные. Преобразуем категориальные признаки (`island`, `sex`) в числа с помощью `pd.get_dummies`. Затем разделим данные на обучающую и тестовую выборки. Обратите внимание, что **масштабирование (`StandardScaler`) не требуется**, так как деревья работают с порогами и нечувствительны к масштабу.*

In [ ]:
from sklearn.model_selection import train_test_split

X = pd.get_dummies(df.drop('species', axis=1), drop_first=True)
y = df['species']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

### 1.2. Модель 1: Одно Дерево Решений

**Комментарий:** *Обучим `DecisionTreeClassifier` с параметрами по умолчанию, чтобы увидеть, на что он способен "из коробки".*

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix

dt_model = DecisionTreeClassifier(random_state=101)
dt_model.fit(X_train, y_train)

dt_preds = dt_model.predict(X_test)

print("--- Результаты Decision Tree ---")
print(classification_report(y_test, dt_preds))

**Комментарий:** *Теперь самое интересное — визуализируем дерево. Оно получилось очень большим и сложным. Это наглядная демонстрация **переобучения**: модель создала очень специфичные правила, чтобы идеально классифицировать каждый объект в обучающей выборке. На новых данных такая сложная модель может работать хуже.*

In [ ]:
from sklearn.tree import plot_tree

plt.figure(figsize=(12, 8))
plot_tree(dt_model, feature_names=X.columns, filled=True)
plt.title("Переобученное дерево решений")
plt.show()

### 1.3. Модель 2: Случайный Лес

**Комментарий:** *Теперь применим `RandomForestClassifier`. Это ансамбль из множества таких деревьев. Ожидаем, что результат будет более стабильным и точным.*

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(n_estimators=100, random_state=101)
rf_model.fit(X_train, y_train)

rf_preds = rf_model.predict(X_test)

print("--- Результаты Random Forest ---")
print(classification_report(y_test, rf_preds))

**Комментарий:** *Случайный лес позволяет оценить **важность признаков** — какой вклад каждый признак вносит в итоговое предсказание. Это усредненная оценка по всем деревьям ансамбля.*

In [ ]:
importances = pd.DataFrame(index=X.columns, data=rf_model.feature_importances_, columns=['Importance'])
importances = importances.sort_values('Importance', ascending=False)
print(importances)

**Вывод по части 1:** *Случайный лес показал более высокую точность, чем одно дерево. Он также дал нам более надежную оценку важности признаков, показав, что длина и высота клюва — самые важные предикторы вида пингвина.*


## Часть 2: Задача Регрессии

**Датасет:** Синтетические данные на основе функции `cos(x)` с добавлением шума.
**Цель:** Увидеть, как дерево и лес аппроксимируют гладкую функцию.

In [ ]:
# Генерируем данные
x_range = np.linspace(0, 10, 100)
noise = np.random.randn(len(x_range)) * 0.2
y_reg = np.cos(x_range) + noise

X_reg = x_range.reshape(-1, 1)

plt.figure(figsize=(10, 6))
plt.scatter(X_reg, y_reg, label='Данные с шумом')
plt.plot(x_range, np.cos(x_range), 'r--', label='Истинная функция cos(x)')
plt.legend()
plt.title("Синтетические данные для задачи регрессии")
plt.show()

### 2.1. Модель 1: Дерево Регрессии

**Комментарий:** *Обучим `DecisionTreeRegressor` с ограниченной глубиной (`max_depth=3`), чтобы его предсказания были наглядными.*

In [ ]:
from sklearn.tree import DecisionTreeRegressor

dt_reg = DecisionTreeRegressor(max_depth=3)
dt_reg.fit(X_reg, y_reg)

dt_reg_preds = dt_reg.predict(X_reg)

**Комментарий:** *Визуализируем результат. Предсказание дерева — это **ступенчатая функция**. Каждый "шаг" соответствует среднему значению в листе дерева.*

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(X_reg, y_reg, alpha=0.5, label='Данные')
plt.plot(x_range, dt_reg_preds, 'g-', label='Предсказания Decision Tree (max_depth=3)')
plt.legend()
plt.title("Аппроксимация деревом регрессии")
plt.show()

### 2.2. Модель 2: Случайный Лес для Регрессии

**Комментарий:** *Теперь обучим `RandomForestRegressor`. Его предсказание будет средним от сотен таких ступенчатых функций.*

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf_reg = RandomForestRegressor(n_estimators=100)
rf_reg.fit(X_reg, y_reg)

rf_reg_preds = rf_reg.predict(X_reg)

**Комментарий:** *Сравним предсказания дерева и леса на одном графике. Предсказание леса гораздо более гладкое и лучше следует за истинной функцией, игнорируя шум.*

In [ ]:
plt.figure(figsize=(12, 8))
plt.scatter(X_reg, y_reg, alpha=0.3, label='Данные')
plt.plot(x_range, dt_reg_preds, 'g-', label='Decision Tree (max_depth=3)', linewidth=2)
plt.plot(x_range, rf_reg_preds, 'm-', label='Random Forest', linewidth=2)
plt.plot(x_range, np.cos(x_range), 'r--', label='Истинная функция cos(x)')
plt.legend()
plt.title("Сравнение Дерева и Случайного Леса в регрессии")
plt.show()

**Итоговый вывод семинара:** *Мы убедились, что древовидные модели можно применять как для классификации, так и для регрессии. В обоих случаях ансамблевый подход в лице Случайного Леса показывает более качественные и стабильные результаты, чем одно дерево решений, эффективно борясь с переобучением.*